In [ ]:
!pip install langchain langchain[ollama] langchain[community]

In [1]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def power(a: int, b: int) -> int:
    """Return a to the power of b """
    return a ** b

In [2]:
from langchain_ollama import ChatOllama
import os

llm = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), format="json", temperature=0)
llm_with_tools = llm.bind_tools(tools=[add, multiply, power])

response = llm_with_tools.invoke("안녕?")
print(response)

query = "2의 100승은?"
response = llm_with_tools.invoke(query)
print(response)

content='' additional_kwargs={} response_metadata={} id='run-fa405cda-47bf-4c28-81d7-4e4b387fe4e0-0' tool_calls=[{'name': 'directly-answer', 'args': {}, 'id': '33de724e-bf86-4adb-aa89-878cd1292529', 'type': 'tool_call'}]
content='' additional_kwargs={} response_metadata={} id='run-390bd53f-c753-4c7b-8b89-e736e8883ee3-0' tool_calls=[{'name': 'power', 'args': {'a': 2, 'b': 100}, 'id': 'a344e389-e828-4cd7-9239-6a8b513a9bbb', 'type': 'tool_call'}]


In [3]:
from langchain_core.messages import HumanMessage, FunctionMessage

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(query)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply, "power": power}[tool_call["name"].lower()]
    tool_output = str(selected_tool.invoke(tool_call["args"]))
    messages.append(FunctionMessage(tool_output, name=tool_call["name"]))

print(messages)

[HumanMessage(content='2의 100승은?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={}, id='run-4acf4f20-39d0-423c-974f-4df1d5712daa-0', tool_calls=[{'name': 'power', 'args': {'a': 2, 'b': 100}, 'id': '3ccb235b-0cec-47b7-91a2-8f98b0e57b03', 'type': 'tool_call'}]), FunctionMessage(content='1267650600228229401496703205376', additional_kwargs={}, response_metadata={}, name='power')]


In [4]:
from langchain_community.llms import Ollama
import os

Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)

/tmp/ipykernel_51097/1621789022.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  Ollama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0).invoke(messages)


'당신이 묻는 것은 2의 100제곱에 대한 계산입니다. 이는 2를 100번 곱하는 것을 의미합니다. 결과는 다음과 같습니다:\n\n2의 100승 = 2^100 = 1267650600228229401496703205376'

In [6]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def process_query(query):
    messages = []
    ai_msg = llm_with_tools.invoke(query)
    messages.append(ai_msg)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"add": add, "multiply": multiply, "power": power }[tool_call["name"].lower()]
        tool_output = str(selected_tool.invoke(tool_call["args"]))
        messages.append(FunctionMessage(tool_output, name=tool_call["name"]))   

    chat_prompt = ChatPromptTemplate.from_template("""다음 내용에 근거하여 질문에 간단히 한국어로 답하세요:
                                                {context}
                                                
                                                Question:{question}
                                                Answer:""")
    chat_model = ChatOllama(model=os.getenv("OPENAI_DEFAULT_MODEL"), temperature=0)
    llm_chain = chat_prompt | chat_model| StrOutputParser()
    response = llm_chain.invoke({"context":messages, "question":query})
    if response:
        return response
    else:
        return "No response found."

query = "23의 17승은 얼마입니까?"
process_query(query)

'23의 17승은 약 1.41 x 10^258입니다.'